In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob 

In [2]:
# Get a list of all .csv's in listings folder 
csv_list = glob.glob('Data/listings/*.csv')
# Load .csv's and create list of df's
dataframes_list = list()
for file in csv_list:
    dataframe = pd.read_csv(file)
    dataframes_list.append(dataframe)

/Users/martabarrios/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (40,85) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/martabarrios/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (40) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/martabarrios/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (43,88) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/martabarrios/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  interacti

In [3]:
concat_df = pd.concat(dataframes_list, ignore_index=True)

In [4]:
dataframe.head()

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,4717459,https://www.airbnb.com/rooms/4717459,20161001085131,2016-10-02,"Large, Modern Apt near Times Square",Large modern studio located in a doorman build...,Large modern studio located in a doorman build...,Large modern studio located in a doorman build...,none,"Hell's Kitchen is located in Midtown West, ste...",...,10.0,f,NaN,NaN,f,strict,f,f,1,0.09
1,9436049,https://www.airbnb.com/rooms/9436049,20161001085131,2016-10-02,Huge room with private bathroom,"Welcome to Brooklyn! The room is huge, has a q...",You will be sharing the apartment with my two ...,"Welcome to Brooklyn! The room is huge, has a q...",none,Bed-Stuy is the the new gem of Brooklyn. Get t...,...,NaN,f,NaN,NaN,f,moderate,f,f,1,NaN
2,6028165,https://www.airbnb.com/rooms/6028165,20161001085131,2016-10-02,BOHEMIAN CHIC* 1BR* OURDOOR AREA,"Queen sized bed, outdoor area, great location.",Located on the border of Chinatown and Lower E...,"Queen sized bed, outdoor area, great location....",none,NaN,...,10.0,f,NaN,NaN,f,strict,f,f,1,0.22
3,5604005,https://www.airbnb.com/rooms/5604005,20161001085131,2016-10-02,Make Yourself At Home NYC,Clean Spacious Modern 'Newly' Renovated 2 Bedr...,Clean Spacious Modern 'Newly' Renovated 2 Bedr...,Clean Spacious Modern 'Newly' Renovated 2 Bedr...,none,"Its in a Very Busy well Lit Area ,Plenty of Re...",...,9.0,f,NaN,NaN,f,moderate,f,f,1,2.06
4,11271036,https://www.airbnb.com/rooms/11271036,20161001085131,2016-10-02,Room in East Harlem,Master bedroom (10x12 ft) in true 3 bedroom / ...,Master bedroom (10x12) in true 3 bedroom / 1.5...,Master bedroom (10x12 ft) in true 3 bedroom / ...,none,NaN,...,10.0,f,NaN,NaN,f,flexible,f,f,1,0.14


# Which columns are we going to use? 

In [5]:
dataframe.shape

(39553, 95)

In [7]:
list(dataframe)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [58]:
dataframe['require_guest_phone_verification'].unique()

array(['f', 't'], dtype=object)

In [ ]:
dataframe.shape[0] - (dataframe['host_listings_count'] == dataframe['host_total_listings_count']).sum()

In [ ]:
dataframe.shape[0]

In [11]:
selected_columns = ['id',
 #'listing_url',
 #'scrape_id',
 'last_scraped',
 #'name',
 #'summary',
 #'space',
 #'description',
 #'experiences_offered',
 #'neighborhood_overview',
 #'notes',
 #'transit',
 #'access',
 #'interaction',
 #'house_rules',
 #'thumbnail_url',
 #'medium_url',
 #'picture_url',
 #'xl_picture_url',
 'host_id',
 #'host_url',
 #'host_name',
 'host_since',
 #'host_location',
 #'host_about',
 'host_response_time',
 'host_response_rate',
 #'host_acceptance_rate',
 'host_is_superhost',
 #'host_thumbnail_url',
 #'host_picture_url',
 #'host_neighbourhood',
 #'host_listings_count',
 #'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 #'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 #'city',
 #'state',
 'zipcode',
 #'market',
 #'smart_location',
 #'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',
 'square_feet',
 'price',
 'weekly_price',
 'monthly_price',
 'security_deposit',
 #'cleaning_fee',
 'guests_included',
 'extra_people',
 'minimum_nights',
 'maximum_nights',
 #'calendar_updated',
 #'has_availability',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 #'calendar_last_scraped',
 'number_of_reviews',
 #'first_review',
 #'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 #'requires_license',
 #'license',
 #'jurisdiction_names',
 'instant_bookable',
 'cancellation_policy',
 'require_guest_profile_picture',
 'require_guest_phone_verification'
 #'calculated_host_listings_count',
 #'reviews_per_month'
]

In [14]:
dataframe.loc[:, selected_columns].shape

(39553, 52)